# HEC-RAS 1D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the HdfBase, HdfUtils, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, and HdfResultsXsec classes to streamline data extraction, processing, and visualization.


In [1]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas', 'pyproj', 'shapely', 'xarray','rtree', 'rasterstats']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr


In [2]:
# Install ras-commander if you are not in a dev environment. 
# install_module(ras-commander)

## Importing ras-commander flexibly (from package or local dev copy)

In [ ]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger
except ImportError:
    # If the import fails, add the parent directory to the Python path
    print("Using Local ras-commander dev libraries")
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger

print("ras_commander imported successfully")

In [ ]:
# Download the BaldEagleCrkMulti2D project from HEC and run plan 01

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
bald_eagle_path = current_dir / "example_projects" / "Balde Eagle Creek"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = bald_eagle_path / "BaldEagle.p01.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    ras_examples = RasExamples()
    ras_examples.extract_project(["Balde Eagle Creek"])

    # Initialize custom Ras object
    bald_eagle = RasPrj()

    # Initialize the RAS project using the custom ras object
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    logging.info(f"Balde Eagle project initialized with folder: {bald_eagle.project_folder}")
    
    logging.info(f"Balde Eagle object id: {id(bald_eagle)}")
    
    # Define the plan number to execute
    plan_number = "01"

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number, ras_object=bald_eagle)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("BaldEagle.p01.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    bald_eagle = RasPrj()
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    plan_number = "01"

In [ ]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for bald_eagle project
print("Plan DataFrame for bald_eagle project:")
display(bald_eagle.plan_df)

# Display geom_df for bald_eagle project
print("\nGeometry DataFrame for bald_eagle project:")
display(bald_eagle.geom_df)

# Get the plan HDF path
plan_number = "01"  # Assuming we're using plan 01 as in the previous code
plan_hdf_path = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

# Get the geometry file number from the plan DataFrame
geom_file = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'Geom File'].values[0]
geom_number = geom_file[1:]  # Remove the 'g' prefix

# Get the geometry HDF path
geom_hdf_path = bald_eagle.geom_df.loc[bald_eagle.geom_df['geom_number'] == geom_number, 'hdf_path'].values[0]

print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")

-----

RasHdfUtils
| Method Name | Description |
|-------------|-------------|
| get_attrs | Converts attributes from a HEC-RAS HDF file into a Python dictionary for a given attribute path |
| get_root_attrs | Returns attributes at root level of HEC-RAS HDF file |
| get_hdf_paths_with_properties | Gets all paths in the HDF file with their properties |
| get_group_attributes_as_df | Gets attributes of a group in the HDF file as a DataFrame |
| get_hdf_filename | Gets the HDF filename from various input types |
| get_runtime_data | Extracts runtime and compute time data from a single HDF file |


In [ ]:
# Get HDF Paths with Properties (For Exploring HDF Files)
HdfBase.get_dataset_info(plan_number, ras_object=bald_eagle, group_path="/Geometry")

In [ ]:
# Use HdfPlan for geometry-related operations
print("\nExample: Extracting Base Geometry Attributes")
geom_attrs = HdfPlan.get_geometry_information(geom_hdf_path, ras_object=bald_eagle)
display(geom_attrs)

In [ ]:
# Example: Extract runtime and compute time data
print("\nExample 2: Extracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_input=plan_number, ras_object=bald_eagle)
if runtime_df is not None:
    display(runtime_df)
else:
    print("No runtime data found.")

Table of all the functions in the RasGeomHdf class from the ras_commander/RasGeomHdf.py file:

| Function Name | Description |
|---------------|-------------|
| projection | Returns the projection of the RAS geometry as a pyproj.CRS object |
| get_geom_attrs | Returns base geometry attributes from a HEC-RAS HDF file |

| mesh_area_names | Returns a list of the 2D mesh area names of the RAS geometry |
| get_geom_2d_flow_area_attrs | Returns geometry 2d flow area attributes from a HEC-RAS HDF file |
| mesh_areas | Returns 2D flow area perimeter polygons |
| mesh_cell_polygons | Returns 2D flow mesh cell polygons |
| mesh_cell_points | Returns 2D flow mesh cell points |
| mesh_cell_faces | Returns 2D flow mesh cell faces |

| get_geom_structures_attrs | Returns geometry structures attributes from a HEC-RAS HDF file |




| bc_lines | Returns 2D mesh area boundary condition lines |
| breaklines | Returns 2D mesh area breaklines |



| refinement_regions | Returns 2D mesh area refinement regions |
| structures | Returns the model structures |
| reference_lines_names | Returns reference line names |
| reference_points_names | Returns reference point names |
| reference_lines | Returns the reference lines geometry and attributes |
| reference_points | Returns the reference points geometry and attributes |
| cross_sections | Returns the model 1D cross sections |
| river_reaches | Returns the model 1D river reach lines |
| cross_sections_elevations | Returns the model cross section elevation information |

In [ ]:
# For all of the RasGeomHdf Class Functions, we will use geom_hdf_path
print(geom_hdf_path)

# For the example project, plan 06 is associated with geometry 09
# If you want to call the geometry by number, call RasHdfGeom functions with a number
# Otherwise, if you want to look up geometry hdf path by plan number, follow the logic in the previous code cells

In [ ]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Projection: {projection}")
else:
    print("No projection information found.  This attribute is only included if a RASMapper projection is defined.")

In [ ]:
# Use HdfPlan for geometry-related operations
print("\nExample: Extracting Base Geometry Attributes")
geom_attrs = HdfPlan.get_geometry_information(geom_hdf_path)
display(geom_attrs)



In [ ]:
# Get geometry structures attributes
print("\nGetting geometry structures attributes")
geom_structures_attrs = HdfStruc.get_geom_structures_attrs(geom_hdf_path, ras_object=bald_eagle)
if geom_structures_attrs:
    print("Geometry structures attributes:")
    for key, value in geom_structures_attrs.items():
        print(f"{key}: {value}")
else:
    print("No geometry structures attributes found.")

### NEED TO EDIT THIS TO SHOW BC LINES WITH RIVERS AND CROSS SECTIONS

# Example: Extract Boundary Condition Lines and Plot with 2D Flow Area Perimeter Polygons
print("\nExample 7: Extracting Boundary Condition Lines and Plotting with 2D Flow Area Perimeter Polygons")
bc_lines_df = HdfBndry.bc_lines(geom_hdf_path, ras_object=bald_eagle)
if not bc_lines_df.empty:
    display(bc_lines_df.head())
else:
    print("No Boundary Condition Lines found.")

# Plot if data exists
if not bc_lines_df.empty or not mesh_areas.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')
    
    # Plot boundary condition lines
    if not bc_lines_df.empty:
        bc_lines_df.plot(ax=ax, color='red', linewidth=2, label='Boundary Condition Lines')
    
    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Boundary Condition Lines')
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No data available for plotting.")

In [13]:
# INSTEAD OF hdf_input, USE plan_hdf_path or geom_hdf_path as appropriate 

In [ ]:
# Example: Get structures
structures_gdf = HdfStruc.get_structures(geom_hdf_path)
print("Structures:")
if not structures_gdf.empty:
    display(structures_gdf.head())
else:
    print("No structures found in the geometry file.")

In [ ]:
# Example: Get reference lines
ref_lines_gdf = HdfBndry.get_reference_lines(geom_hdf_path, ras_object=bald_eagle)
print("\nReference Lines:")
if not ref_lines_gdf.empty:
    display(ref_lines_gdf.head())
else:
    print("No reference lines found in the geometry file.")

In [ ]:
# Example: Get reference points
ref_points_gdf = HdfBndry.get_reference_points(geom_hdf_path, ras_object=bald_eagle)
print("\nReference Points:")
if not ref_points_gdf.empty:
    display(ref_points_gdf.head())
else:
    print("No reference points found in the geometry file.")

In [ ]:
# Use the get_hdf5_dataset_info function from HdfUtils to explore the Cross Sections structure in the geometry HDF file

print("\nExploring Cross Sections structure in geometry file:")
print("HDF Base Path: /Geometry/Cross Sections ")
HdfBase.get_dataset_info(geom_hdf_path, group_path='/Geometry/Cross Sections')


In [ ]:
# Example: Get cross section geodataframe
cross_sections_gdf = HdfXsec.get_cross_sections(geom_hdf_path, ras_object=bald_eagle)
with pd.option_context('display.max_columns', None):  # Show all columns
    display(cross_sections_gdf)

cross_sections_gdf: 

| geometry | station_elevation | mannings_n | ineffective_blocks | River | Reach | RS | Name | Description | Len Left | Len Channel | Len Right | Left Bank | Right Bank | Friction Mode | Contr | Expan | Left Levee Sta | Left Levee Elev | Right Levee Sta | Right Levee Elev | HP Count | HP Start Elev | HP Vert Incr | HP LOB Slices | HP Chan Slices | HP ROB Slices | Ineff Block Mode | Obstr Block Mode | Default Centerline | Last Edited |
|-----------|-------------------|------------|--------------------|-------|-------|----|------|-------------|----------|-------------|-----------|-----------|------------|----------------|-------|-------|----------------|-----------------|----------------|------------------|----------|----------------|---------------|----------------|----------------|----------------|------------------|------------------|-------------------|--------------|
| 0         | LINESTRING (1968668.17 290166.79, 1969067.87 2... | [[0.0, 660.41], [5.0, 660.61], [40.0, 659.85],... | {'Station': [0.0, 190.0, 375.0], 'Mann n': [0.... | []    | Bald Eagle | Loc Hav | 138154.4 |             | 358.429993 | 463.640015 | 517.640015 | 190.000000 | 375.000000 | Basic Mann n | 0.1   | 0.3   | NaN            | NaN             | NaN            | NaN              | 49       | 656.799988      | 1.0           | 5              | 5              | 5              | 0                | 0                | 0                 | 18Sep2000 09:10:52 |
| 1         | LINESTRING (1968627.02 290584.12, 1969009.09 2... | [[0.0, 664.28], [50.0, 661.73], [55.0, 661.54]... | {'Station': [0.0, 535.0, 672.5599975585938], '... | []    | Bald Eagle | Loc Hav | 137690.8 |             | 305.709991 | 363.839996 | 382.829987 | 535.000000 | 672.559998 | Basic Mann n | 0.1   | 0.3   | NaN            | NaN             | NaN            | NaN              | 65       | 654.229980      | 1.0           | 5              | 5              | 5              | 0                | 0                | 0                 | 18Sep2000 09:10:52 |
| 2         | LINESTRING (1968585.88 290854.5, 1968868.02 29... | [[0.0, 662.72], [20.0, 665.5], [25.0, 666.48],... | {'Station': [0.0, 580.0, 717.239990234375], 'M... | []    | Bald Eagle | Loc Hav | 137327.0 |             | 732.929993 | 762.020020 | 765.359985 | 580.000000 | 717.239990 | Basic Mann n | 0.1   | 0.3   | NaN            | NaN             | NaN            | NaN              | 66       | 653.900024      | 1.0           | 5              | 5              | 5              | 0                | 0                | 0                 | 18Sep2000 09:10:52 |


In [ ]:
# Filter rows where ineffective_blocks is not empty
ineffective_xs = cross_sections_gdf[cross_sections_gdf['ineffective_blocks'].apply(len) > 0]

print("\nCross Sections with Ineffective Flow Areas:")
display(ineffective_xs)

# Print a message if no cross sections with ineffective flow areas are found
print("\nNo cross sections found with ineffective flow areas." if ineffective_xs.empty else "")


In [ ]:
# Print cross sections data

print("\nCross Section Information:")
if not cross_sections_gdf.empty:
    for idx, row in cross_sections_gdf.iterrows():
        print(f"\nCross Section {idx + 1}:")
        print(f"River: {row['River']}")
        print(f"Reach: {row['Reach']}")
        print("\nGeometry:")
        print(row['geometry'])
        print("\nStation-Elevation Points:")
        
        # Print header
        print("     #      Station   Elevation        #      Station   Elevation        #      Station   Elevation        #      Station   Elevation        #      Station   Elevation")
        print("-" * 150)
        
        # Calculate number of rows needed
        points = row['station_elevation']
        num_rows = (len(points) + 4) // 5  # Round up division
        
        # Print points in 5 columns
        for i in range(num_rows):
            line = ""
            for j in range(5):
                point_idx = i + j * num_rows
                if point_idx < len(points):
                    station, elevation = points[point_idx]
                    line += f"{point_idx+1:6d} {station:10.2f} {elevation:10.2f}    "
            print(line)
        print("-" * 150)
else:
    print("No cross sections found in the geometry file.")


In [ ]:
# Plot cross sections on map
import matplotlib.pyplot as plt

# Get cross sections data
cross_sections_gdf = HdfXsec.get_cross_sections(geom_hdf_path, ras_object=bald_eagle)

if not cross_sections_gdf.empty:
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(15,10))
    
    # Plot cross sections
    cross_sections_gdf.plot(ax=ax, color='red', linewidth=1, label='Cross Sections')
    
    # Add river name and reach labels
    #for idx, row in cross_sections_gdf.iterrows():
    #    # Get midpoint of cross section line for label placement
    #    midpoint = row.geometry.centroid
    #    label = f"{row['River']}\n{row['Reach']}\nRS: {row['RS']}"
    #    ax.annotate(label, (midpoint.x, midpoint.y), 
    #               xytext=(5, 5), textcoords='offset points',
    #               fontsize=8, bbox=dict(facecolor='white', alpha=0.7))
    
    # Customize plot
    ax.set_title('Cross Sections Location Map')
    ax.grid(True)
    ax.legend()
    
    # Equal aspect ratio to preserve shape
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
else:
    print("No cross sections found in the geometry file.")



In [ ]:
# Plot cross sections with Manning's n values colored by value
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString

# Create figure
fig, ax1 = plt.subplots(figsize=(20,10))

# Create colormap
cmap = plt.cm.viridis
norm = plt.Normalize(vmin=0.02, vmax=0.08)  # Typical Manning's n range

# Plot cross sections colored by Manning's n
for idx, row in cross_sections_gdf.iterrows():
    # Extract Manning's n values and stations
    mannings = row['mannings_n']
    n_values = mannings['Mann n']
    stations = mannings['Station']
    
    # Get the full linestring coordinates
    line_coords = list(row.geometry.coords)
    
    # Calculate total length of the cross section
    total_length = row.geometry.length
    
    # For each Manning's n segment
    for i in range(len(n_values)-1):
        # Calculate the start and end proportions along the line
        start_prop = stations[i] / stations[-1]
        end_prop = stations[i+1] / stations[-1]
        
        # Get the start and end points for this segment
        start_idx = int(start_prop * (len(line_coords)-1))
        end_idx = int(end_prop * (len(line_coords)-1))
        
        # Extract the segment coordinates
        segment_coords = line_coords[start_idx:end_idx+1]
        
        if len(segment_coords) >= 2:
            # Create a line segment
            segment = LineString(segment_coords)
            
            # Get color from colormap for this n value
            color = cmap(norm(n_values[i]))
            
            # Plot the segment
            ax1.plot(*segment.xy, color=color, linewidth=2)

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, ax=ax1, label="Manning's n Value")

ax1.set_title("Cross Sections Colored by Manning's n Values")
ax1.grid(True)
ax1.set_aspect('equal')

plt.tight_layout()
plt.show()



In [ ]:
# Plot cross sections with ineffective flow areas
import matplotlib.pyplot as plt

# Get cross sections data
cross_sections_gdf = HdfXsec.get_cross_sections(geom_hdf_path, ras_object=bald_eagle)

# Create figure
fig, ax2 = plt.subplots(figsize=(20,10))

# Plot all cross sections first
cross_sections_gdf.plot(ax=ax2, color='lightgray', linewidth=1, label='Cross Sections')

# Plot ineffective flow areas with thicker lines
ineffective_sections = cross_sections_gdf[cross_sections_gdf['ineffective_blocks'].apply(lambda x: len(x) > 0)]
ineffective_sections.plot(ax=ax2, color='red', linewidth=3, label='Ineffective Flow Areas')

# Add ineffective flow area labels with offset to lower right
for idx, row in cross_sections_gdf.iterrows():
    # Get midpoint of cross section line
    midpoint = row.geometry.centroid
    
    # Extract ineffective flow blocks
    ineff_blocks = row['ineffective_blocks']
    
    if ineff_blocks:  # Only label if there are ineffective blocks
        label_parts = []
        # Add RS to first line of label
        label_parts.append(f"RS: {row['RS']}")
        for block in ineff_blocks:
            label_parts.append(
                f"L:{block['Left Sta']:.0f}-R:{block['Right Sta']:.0f}\n"
                f"Elev: {block['Elevation']:.2f}\n"
                f"Permanent: {block['Permanent']}"
            )
        
        label = '\n'.join(label_parts)
        
        ax2.annotate(label, (midpoint.x, midpoint.y),
                    xytext=(15, -15),  # Offset to lower right
                    textcoords='offset points',
                    fontsize=8, 
                    bbox=dict(facecolor='white', alpha=0.7),
                    arrowprops=dict(arrowstyle='->'),
                    horizontalalignment='left',
                    verticalalignment='top')

ax2.set_title('Cross Sections with Ineffective Flow Areas')
ax2.grid(True)
ax2.legend()
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

In [ ]:
# Plot cross section elevation for cross section 42
import matplotlib.pyplot as plt
import numpy as np

# Get cross sections data
cross_sections_gdf = HdfXsec.get_cross_sections(geom_hdf_path, ras_object=bald_eagle)

if not cross_sections_gdf.empty:
    # Get station-elevation data for cross section 42
    station_elevation = cross_sections_gdf.iloc[42]['station_elevation']
    
    # Convert list of lists to numpy arrays for plotting
    stations = np.array([point[0] for point in station_elevation])
    elevations = np.array([point[1] for point in station_elevation])
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12,8))
    
    # Plot cross section
    ax.plot(stations, elevations, 'b-', linewidth=2)
    
    # Add labels and title
    river = cross_sections_gdf.iloc[42]['River']
    reach = cross_sections_gdf.iloc[42]['Reach'] 
    rs = cross_sections_gdf.iloc[42]['RS']
    
    # Show bank stations as dots
    left_bank_station = cross_sections_gdf.iloc[42]['Left Bank']
    right_bank_station = cross_sections_gdf.iloc[42]['Right Bank']
    
    # Interpolating bank stations for plotting
    ax.plot(left_bank_station, elevations[np.searchsorted(stations, left_bank_station)], 'ro', label='Left Bank Station')
    ax.plot(right_bank_station, elevations[np.searchsorted(stations, right_bank_station)], 'ro', label='Right Bank Station')
    
    ax.set_title(f'Cross Section Profile\nRiver: {river}, Reach: {reach}, RS: {rs}\n'
                 f'Left Bank Station: {left_bank_station}, Right Bank Station: {right_bank_station}')
    ax.set_xlabel('Station (ft)')
    ax.set_ylabel('Elevation (ft)')
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No cross sections found in the geometry file.")


In [ ]:
# Example usage:
centerlines = HdfXsec.get_river_centerlines(geom_hdf_path)
centerlines_with_stations = HdfXsec.get_river_stationing(centerlines)

# Display results
print("\nRiver Centerlines:")
display(centerlines.head())

In [ ]:
# Plot river centerlines with labels
import matplotlib.pyplot as plt
import geopandas as gpd

# Create figure and axis
fig, ax = plt.subplots(figsize=(15, 10))

# Plot centerlines
centerlines.plot(ax=ax, color='blue', linewidth=2, label='River Centerline')

# Add river/reach labels
for idx, row in centerlines.iterrows():
    # Get midpoint of the line for label placement
    midpoint = row.geometry.interpolate(0.5, normalized=True)
    
    # Create label text combining river and reach names
    label = f"{row['River Name']}\n{row['Reach Name']}"
    
    # Add text annotation
    ax.annotate(label, 
                xy=(midpoint.x, midpoint.y),
                xytext=(10, 10), # Offset text slightly
                textcoords='offset points',
                fontsize=10,
                bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

# Add labels and title
ax.set_title('River Centerlines', fontsize=14)
ax.set_xlabel('Easting', fontsize=12)
ax.set_ylabel('Northing', fontsize=12)

# Add legend
ax.legend(fontsize=12)

# Add grid
ax.grid(True)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()



In [ ]:
# Example usage:
edge_lines = HdfXsec.get_river_edge_lines(geom_hdf_path)
centerlines = HdfXsec.get_river_centerlines(geom_hdf_path)
# Display results
print("\nRiver Edge Lines:")
display(edge_lines.head())

In [ ]:
# Example usage:
bank_lines = HdfXsec.get_river_bank_lines(geom_hdf_path)
# Display results
print("\nRiver Bank Lines:")
display(bank_lines.head())

In [ ]:
# Create figure and axis
fig, ax = plt.subplots(figsize=(15, 10))

# Plot river edge lines
edge_lines.plot(ax=ax, color='blue', linewidth=2, label='River Edge Lines')

# Plot centerlines for reference
centerlines.plot(ax=ax, color='red', linewidth=2, linestyle='--', label='River Centerline')

# Plot river bank lines
bank_lines.plot(ax=ax, color='green', linewidth=2, label='River Bank Lines')

# Add title and labels
ax.set_title('River Edge Lines, Centerline, and Bank Lines', fontsize=14)
ax.set_xlabel('Easting', fontsize=12)
ax.set_ylabel('Northing', fontsize=12)

# Add legend
ax.legend(fontsize=12)

# Add grid
ax.grid(True)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# Use get_hdf5_dataset_info function to get dataset structure:
HdfBase.get_dataset_info(plan_hdf_path, "/Geometry/River Bank Lines/")

## Function to explore HDF file to assist with 1D Structures Data Extraction 

In [ ]:
# Use get_hdf5_dataset_info function to get Pipe Conduits data:
HdfBase.get_dataset_info(plan_hdf_path, "/Results/Unsteady/Output/Output Blocks/Computation Block/Global/")

In [ ]:
# Use get_hdf5_dataset_info function to get Pipe Conduits data:
HdfBase.get_dataset_info(plan_hdf_path, "/Geometry/Structures")


In [ ]:
# Extract 1D Structures Geodataframe

# Import required functions
from ras_commander.HdfStruc import HdfStruc
from ras_commander.HdfXsec import HdfXsec

# Create instances
hdf_struc = HdfStruc()
hdf_xsec = HdfXsec()

# Extract data into GeoDataFrames
structures_gdf = hdf_struc.get_structures(hdf_path=geom_hdf_path)
cross_sections_gdf = hdf_xsec.get_cross_sections(hdf_path=geom_hdf_path)
centerlines_gdf = hdf_xsec.get_river_centerlines(hdf_path=geom_hdf_path)

# Display basic information about the structures
print("\nStructures Summary:")
print(f"Number of structures found: {len(structures_gdf)}")
display(structures_gdf)

# Display first few rows of key attributes
print("\nStructure Details:")
display_cols = ['Structure ID', 'Structure Type', 'River Name', 'Reach Name', 'Station']
display_cols = [col for col in display_cols if col in structures_gdf.columns]
if display_cols:
    print(structures_gdf[display_cols].head())

# Create visualization
fig, ax = plt.subplots(figsize=(15, 10))

# Plot river centerlines
if not centerlines_gdf.empty:
    centerlines_gdf.plot(ax=ax, color='blue', linewidth=2, label='River Centerlines')

# Plot cross sections
if not cross_sections_gdf.empty:
    cross_sections_gdf.plot(ax=ax, color='green', linewidth=1, label='Cross Sections')

# Plot structures
if not structures_gdf.empty:
    structures_gdf.plot(ax=ax, color='red', marker='s', markersize=100, label='Structures')

# Add title and labels
ax.set_title('HEC-RAS Model Components', fontsize=14)
ax.set_xlabel('Easting', fontsize=12)
ax.set_ylabel('Northing', fontsize=12)

# Add legend
ax.legend(fontsize=12)

# Add grid
ax.grid(True)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()

# Print summary of cross sections
print("\nCross Sections Summary:")
print(f"Number of cross sections found: {len(cross_sections_gdf)}")
if not cross_sections_gdf.empty:
    print("\nCross Section Details:")
    xs_display_cols = ['River', 'Reach', 'Station']
    xs_display_cols = [col for col in xs_display_cols if col in cross_sections_gdf.columns]
    if xs_display_cols:
        print(cross_sections_gdf[xs_display_cols].head())


In [ ]:
# Extract Compute Messages as String
print("Extracting Compute Messages")

import h5py
import numpy as np

def extract_string_from_hdf(results_hdf_filename: str, hdf_path: str) -> str:
    """
    Extract string from HDF object at a given path

    Parameters
    ----------
    results_hdf_filename : str
        Name of the HDF file
    hdf_path : str
        Path of the object in the HDF file

    Returns
    -------
    str
        Extracted string from the specified HDF object
    """
    with h5py.File(results_hdf_filename, 'r') as hdf_file:
        try:
            hdf_object = hdf_file[hdf_path]
            if isinstance(hdf_object, h5py.Group):
                return f"Group: {hdf_path}\nContents: {list(hdf_object.keys())}"
            elif isinstance(hdf_object, h5py.Dataset):
                data = hdf_object[()]
                if isinstance(data, bytes):
                    return data.decode('utf-8')
                elif isinstance(data, np.ndarray) and data.dtype.kind == 'S':
                    return [v.decode('utf-8') for v in data]
                else:
                    return str(data)
            else:
                return f"Unsupported object type: {type(hdf_object)}"
        except KeyError:
            return f"Path not found: {hdf_path}"

try:
    results_summary_string = extract_string_from_hdf(plan_hdf_path, '/Results/Summary/Compute Messages (text)')
    print("Compute Messages:")
    
    # Parse and print the compute messages in a more visually friendly way
    messages = results_summary_string[0].split('\r\n')
    
    for message in messages:
        if message.strip():  # Skip empty lines
            if ':' in message:
                key, value = message.split(':', 1)
                print(f"{key.strip():40} : {value.strip()}")
            else:
                print(f"\n{message.strip()}")
    
    # Print computation summary in a table format
    print("\nComputation Summary:")
    print("-" * 50)
    print(f"{'Computation Task':<30} {'Time':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Task' in line:
            task, time = line.split('\t')
            print(f"{task:<30} {time:<20}")
    
    print("\nComputation Speed:")
    print("-" * 50)
    print(f"{'Task':<30} {'Simulation/Runtime':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Speed' in line:
            task, speed = line.split('\t')
            print(f"{task:<30} {speed:<20}")

except Exception as e:
    print(f"Error extracting compute messages: {str(e)}")
    print("\nNote: If 'Results/Summary Output' is not in the file structure, it might indicate that the simulation didn't complete successfully or the results weren't saved properly.")

 



In [ ]:
# Example 12: Extract Plan Parameters and Volume Accounting
print("\nExample 12: Extracting Plan Parameters and Volume Accounting Data")

# Extract plan parameters
plan_parameters_df = HdfPlan.get_plan_parameters(hdf_path=plan_hdf_path)

# Extract volume accounting data
volume_accounting_df = HdfResultsPlan.get_volume_accounting(hdf_path=plan_hdf_path)

print("\nPlan Parameters DataFrame:")
display(plan_parameters_df)

print("\nVolume Accounting DataFrame:")
display(volume_accounting_df)

# RasPlanHdf Class Functions

In [ ]:
# Example: Get simulation start time
start_time = HdfPlan.get_plan_start_time(plan_hdf_path)
print(f"Simulation start time: {start_time}")

In [ ]:
# Example: Get plan end time
end_time = HdfPlan.get_plan_end_time(plan_hdf_path)
print(f"Simulation end time: {end_time}")

## Modify the cell below to time of max wsel for 1D models

In [ ]:
# Plot the time of maximum water surface elevation (WSEL) for cross sections
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# Get cross section results timeseries
xsec_results = HdfResultsXsec.get_xsec_timeseries(plan_hdf_path)
print("\nCross Section Results Shape:", xsec_results['Water_Surface'].shape)

# Get cross section geometry data
xsec_geom = HdfXsec.get_cross_sections(plan_hdf_path)
print("\nNumber of cross sections in geometry:", len(xsec_geom))

# Create dataframe with cross section locations and max WSEL times
xs_data = []

# Extract water surface data from xarray Dataset
water_surface = xsec_results['Water_Surface'].values
times = pd.to_datetime(xsec_results.time.values)

# Debug print
print("\nFirst few cross section names:")
print(xsec_results.cross_section.values[:5])

# Iterate through cross sections
for xs_idx in range(len(xsec_results.cross_section)):
    # Get WSEL timeseries for this cross section
    wsel_series = water_surface[:, xs_idx]
    
    # Get cross section name and parse components
    xs_name = xsec_results.cross_section.values[xs_idx]
    
    # Split the string and remove empty strings
    xs_parts = [part for part in xs_name.split() if part]
    
    if len(xs_parts) >= 3:
        river = "Bald Eagle"  # Combine first two words
        reach = "Loc Hav"     # Next two words
        rs = xs_parts[-1]     # Last part is the station
        
        # Get geometry for this cross section
        xs_match = xsec_geom[
            (xsec_geom['River'] == river) & 
            (xsec_geom['Reach'] == reach) & 
            (xsec_geom['RS'] == rs)
        ]
        
        if not xs_match.empty:
            geom = xs_match.iloc[0]
            # Use first point of cross section line for plotting
            x = geom.geometry.coords[0][0]
            y = geom.geometry.coords[0][1]
            
            # Find time of max WSEL
            max_wsel_idx = np.argmax(wsel_series)
            max_wsel = np.max(wsel_series)
            max_time = times[max_wsel_idx]
            
            xs_data.append({
                'xs_name': xs_name,
                'x': x,
                'y': y,
                'max_wsel': max_wsel,
                'time_of_max': max_time
            })
        else:
            print(f"\nWarning: No geometry match found for {xs_name}")
            print(f"River: {river}, Reach: {reach}, RS: {rs}")
    else:
        print(f"\nWarning: Could not parse cross section name: {xs_name}")

# Create dataframe
xs_df = pd.DataFrame(xs_data)

# Debug print
print("\nNumber of cross sections processed:", len(xs_df))
if not xs_df.empty:
    print("\nColumns in xs_df:", xs_df.columns.tolist())
    print("\nFirst row of xs_df:")
    print(xs_df.iloc[0])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Convert datetime to hours since start for colormap
    min_time = min(xs_df['time_of_max'])
    color_values = [(t - min_time).total_seconds() / 3600 for t in xs_df['time_of_max']]

    # Plot cross section points
    scatter = ax.scatter(xs_df['x'], xs_df['y'],
                        c=color_values,
                        cmap='viridis',
                        s=50)

    # Customize plot
    ax.set_title('Time of Maximum Water Surface Elevation at Cross Sections')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Add colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Hours since simulation start')

    # Format colorbar ticks
    max_hours = int(max(color_values))
    tick_interval = max(1, max_hours // 6)  # Show ~6 ticks
    cbar.set_ticks(range(0, max_hours + 1, tick_interval))
    cbar.set_ticklabels([f'{h}h' for h in range(0, max_hours + 1, tick_interval)])

    # Add grid and adjust styling
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.rcParams.update({'font.size': 12})
    plt.tight_layout()

    # Show plot
    plt.show()

    # Print summary statistics
    max_wsel_xs = xs_df.loc[xs_df['max_wsel'].idxmax()]
    hours_since_start = (max_wsel_xs['time_of_max'] - min_time).total_seconds() / 3600

    print(f"\nOverall Maximum WSEL: {max_wsel_xs['max_wsel']:.2f} ft")
    print(f"Time of Overall Maximum WSEL: {max_wsel_xs['time_of_max']}")
    print(f"Hours since simulation start: {hours_since_start:.2f} hours")
    print(f"Location of Overall Maximum WSEL: X={max_wsel_xs['x']:.2f}, Y={max_wsel_xs['y']:.2f}")
    print(f"Cross Section: {max_wsel_xs['xs_name']}")
else:
    print("\nWarning: No cross sections were processed successfully")
    print("xs_data length:", len(xs_data))

### Need to add this to the ras-commander library

In [ ]:
# Get unsteady attributes
results_unsteady_attrs = HdfResultsPlan.get_unsteady_info(plan_hdf_path, ras_object=bald_eagle)
print("\nResults Unsteady Attributes:")
for key, value in results_unsteady_attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Get unsteady summary attributes
results_unsteady_summary_attrs = HdfResultsPlan.get_unsteady_summary(plan_hdf_path, ras_object=bald_eagle)
print("\nResults Unsteady Summary Attributes:")
for key, value in results_unsteady_summary_attrs.items():
    print(f"{key}: {value}")

# Get volume accounting attributes
volume_accounting_attrs = HdfResultsPlan.get_volume_accounting(plan_hdf_path, ras_object=bald_eagle)
print("\nVolume Accounting Attributes:")
for key, value in volume_accounting_attrs.items():
    print(f"{key}: {value}")

In [ ]:
print("\n=== HDF5 File Structure ===\n")
print(plan_hdf_path)
HdfBase.get_dataset_info(plan_hdf_path, group_path='/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Cross Sections')

In [ ]:
xsec_results = HdfResultsXsec.get_xsec_timeseries(plan_hdf_path)
print(xsec_results)

In [ ]:
# Print time series for specific cross section
target_xs = "Bald Eagle       Loc Hav          136202.3"

print("\nTime Series Data for Cross Section:", target_xs)
for var in ['Water_Surface', 'Velocity_Total', 'Velocity_Channel', 'Flow_Lateral', 'Flow']:
    print(f"\n{var}:")
    print(xsec_results[var].sel(cross_section=target_xs).values[:5])  # Show first 5 values

# Create time series plots
import matplotlib.pyplot as plt

# Create a figure for each variable
variables = ['Water_Surface', 'Velocity_Total', 'Velocity_Channel', 'Flow_Lateral', 'Flow']

for var in variables:
    plt.figure(figsize=(10, 5))
    # Convert time values to datetime if needed
    time_values = pd.to_datetime(xsec_results.time.values)
    values = xsec_results[var].sel(cross_section=target_xs).values
    
    # Plot with explicit x and y values
    plt.plot(time_values, values, '-', linewidth=2)
    
    plt.title(f'{var} at {target_xs}')
    plt.xlabel('Time')
    plt.ylabel(var.replace('_', ' '))
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Force display
    plt.draw()
    plt.pause(0.1)
    plt.show()
